最常见的一种矩阵分解技术就是SVD。SVD将原始的数据集矩阵 data分解为三个矩阵U,$\sigma$,$V^T$.
$$Data_{mxn}=U_{m*m} \sigma_{m*n}V^T_{n*n}$$

In [1]:
from numpy import *

In [2]:
def loadExData():
    return [
            [1,1,1,0,0],
            [2,2,2,0,0],
            [1,1,1,0,0],
            [5,5,5,0,0],
            [1,1,0,2,2],
            [0,0,0,3,3],
            [0,0,0,1,1]
    ]

In [3]:
Data = loadExData()
U,Sigma,VT = linalg.svd(Data)
Sigma

array([  9.72140007e+00,   5.29397912e+00,   6.84226362e-01,
         1.25958571e-15,   2.05750739e-31])

前三个值比其他值大了很多，可以认为是噪声
$$Data_{m*n} = U_{m*3}\sigma_{3*3}V^T_{3*n}$$

In [4]:
# 还原矩阵

In [5]:
Sig3 = mat([[Sigma[0],0,0],[0,Sigma[1],0],[0,0,Sigma[2]]])
Sig3

matrix([[ 9.72140007,  0.        ,  0.        ],
        [ 0.        ,  5.29397912,  0.        ],
        [ 0.        ,  0.        ,  0.68422636]])

In [6]:
U[:,:3]*Sig3*VT[:3,:]

matrix([[  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
           2.56622790e-16,   2.46764414e-16],
        [  2.00000000e+00,   2.00000000e+00,   2.00000000e+00,
           7.09097174e-17,   5.03069808e-17],
        [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          -5.29749231e-16,  -5.36896916e-16],
        [  5.00000000e+00,   5.00000000e+00,   5.00000000e+00,
           1.67297553e-16,   1.29236899e-16],
        [  1.00000000e+00,   1.00000000e+00,  -8.66411359e-16,
           2.00000000e+00,   2.00000000e+00],
        [  4.32342232e-16,   1.03764899e-15,  -1.40367906e-15,
           3.00000000e+00,   3.00000000e+00],
        [  1.31838984e-16,   3.33066907e-16,  -4.44089210e-16,
           1.00000000e+00,   1.00000000e+00]])

# 推荐应用

本次样本数据是11*11，其中行表示用户，列表示食品，中间数字表示该用户对食品的打分。如果数字为0，表示该用户没有吃过该食品。本次模型的目的就是向用户推荐未吃过的食品。

推荐思路：
* 首先，寻找用户未评价的食品，即用户-矩阵中的0值；
* 再次，对用户未打分的食品，通过相似度计算预计其可能会打多少分数；
* 最后，对这些打分的食品根据评分从高到低进行排序，返回前N个食品，这就是推荐结果。


In [7]:
def loadExData2():
    return[[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
           [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
           [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
           [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
           [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
           [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
           [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
           [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
           [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
           [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
           [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0]]

## 基于协同过滤的推荐引擎

### 相似度计算

In [8]:
# inA inB都是列向量
from numpy import *
from numpy import linalg as la
def ecludSim(inA,inB):
    #为将距离映射到[0,1]中，相似度=1/(1+欧氏距离)
    return 1.0/(1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    #皮尔森系数在[-1,1]之间，为映射到[0,1]之间，相似度=0.5+0.5*corroef
    if len(inA) < 3 : return 1.0
    return 0.5+0.5*corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA,inB):
    #余弦夹角在[-1,1]之间，为映射到[0,1]之间，相似度=0.5+0.5*cos
    num = float(inA.T*inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5+0.5*(num/denom)

In [9]:
mymat = mat(loadExData2())
print(ecludSim(mymat[:,0],mymat[:,4]))
print(pearsSim(mymat[:,0],mymat[:,4]))
print(cosSim(mymat[:,0],mymat[:,4]))

0.0846263260896
0.300354087029
0.5


### 基于物品相似度的推荐引擎


In [10]:
#该函数用来计算在给定相似度计算方法的条件下，用户对物品的估计评分值。
def standEst(dataMat,user,simMeas,item):
    n = shape(dataMat)[1]
    simTotal = 0.0;ratSimTotal =0.0
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0 : continue
        #寻找两个用户都评级的物品    
        overLap  = nonzero(logical_and(dataMat[:,item].A>0,\
                          dataMat[:,j].A>0))[0]  
        if len(overLap) ==0:similarity=0
        else:
            similarity = simMeas(dataMat[overLap,item],
                                dataMat[overLap,j])
        simTotal +=similarity
        ratSimTotal +=similarity*userRating
    if simTotal==0:return 0
    else:
        return ratSimTotal/simTotal

### 基于SVD的评分估计

In [11]:
#该函数对给定用户给定物品构建了一个评分估计值。第三行对数据集进行了SVD分解，利用包含了90%能量值的
#奇异值。然后利用这些奇异值构造出对角矩阵，利用U矩阵将物品转换到低维空间中。
def svdEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    U,Sigma,VT = la.svd(dataMat)
    Sig4 = mat(eye(4)*Sigma[:4]) #构建对角矩阵
    xformedItems = dataMat.T * U[:,:4] * Sig4.I  #构建转换后的物品
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0 or j==item: continue
        similarity = simMeas(xformedItems[item,:].T,\
                             xformedItems[j,:].T)
#         print('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal

In [12]:
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=standEst):
    unratedItems = nonzero(dataMat[user,:].A==0)[1]#find unrated items 
    if len(unratedItems) == 0: return 'you rated everything'
    itemScores = []
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]

### 示例：对第三行用户进行推荐

In [13]:
recommend(mymat,2,estMethod=standEst)

[(3, 4.0), (5, 4.0), (6, 4.0)]

输出结果为（[(3, 4.0), (5, 4.0), (6, 4.0)]），推荐第4/5/6个食品。

In [14]:
recommend(mymat,2,estMethod=svdEst)

[(6, 3.0394902391812892), (5, 3.0090087051508894), (3, 3.0058579857590901)]

输出结果为（[(6, 3.04), (5, 3.009), (10, 3.00)]），推荐第7/6/11个食品。

从结果来看SVD推荐效果比较好,因为他对数据噪音进行了处理。

### 进一步理解

数据处理过程中得到的三个矩阵也有相关物理含义。但需要自己在业务中去摸索和理解。

In [15]:
u,s,v = linalg.svd(mymat)

本次样本数据U的前第1列表示的就是用户吃过的食品的个数.两个值越接近表示越相同，
其中第2个和8个用户都是＞ -0.1，表示只吃过2个物品。

In [16]:
u[:,:2]

matrix([[-0.02173672, -0.41043862],
        [-0.01664767, -0.40868796],
        [-0.03763173, -0.27302481],
        [-0.3928286 ,  0.03215633],
        [-0.68146521,  0.05125169],
        [-0.01031581, -0.35826614],
        [-0.60364271, -0.00222591],
        [-0.02078959, -0.4841342 ],
        [-0.01290907, -0.35922701],
        [-0.00900549, -0.30733798],
        [-0.11812788,  0.00805012]])

V的第1行表示个食品之间的联系，比如都是日式、印式、美式等主题概念。

In [17]:
v[:2,]

matrix([[-0.45137416, -0.36239706, -0.46879252, -0.01007685, -0.01567036,
         -0.01664563, -0.00474684, -0.46712774, -0.47223188, -0.01591788,
         -0.0552444 ],
        [ 0.03084799,  0.02584428,  0.03296133, -0.34024331, -0.38750193,
         -0.52000097, -0.18887149,  0.00389831,  0.02853952, -0.39205093,
         -0.52034959]])